In [ ]:
###关于数据的保存：在bash运行——./save.sh "这里写修改说明"###

#===========================================================#
#####################总体思路梳理############################
#===========================================================#
##################模块一：得到目标论文序列####################
#STEP1————匹配
#将论文引用关系原始数据与撤稿论文序列进行匹配，目的是得到所有引用了撤稿论文的论文




# 前期准备模块

## 库的导入

In [3]:
###########################这一单元模块专门用于导入需要用到的库####################################

import pandas as pd #基础的数据处理
import os 
#Python 中的一条指令，用于导入内置的os模块。
#os模块提供了与操作系统交互的功能，允许你执行文件和目录的操作
#如路径管理、文件创建、删除等
import json #用于处理json数据的库
from tqdm import tqdm
import math # 用于检测 NaN
from collections import Counter #Python标准库collections里的的工具，用来统计元素出现的次数
from collections import defaultdict 
'''
collections 是 Python 自带的一个模块，
里面提供了许多有用的容器数据类型，比如 Counter、deque、namedtuple、defaultdict 等。
defaultdict 是一个字典的子类，它和普通字典类似，
但当你访问一个不存在的键时，它会自动帮你创建一个默认值，而不是抛出 KeyError。
'''
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import ast

## 查看大体量数据的前十行（CSV/EXCEL/JSON）

In [ ]:
###########数据前10行展示##############
######用来判断太大数据的数据结构########
#CSV文件：
file_csv = '/Data1/DATA/Dimensions2024/20240101/Papers.csv'
df_csv = pd.read_csv(file_csv, nrows=10)
df_csv

In [ ]:
#EXCEL文件：
file_excel = ''
df_excel = pd.read_excel(file_excel, nrows=10)
df_excel

In [ ]:
#JSON文件：
file_json = 'JSON文件路径.json'
df_json = pd.read_json(file_json)
df_json

## 查看CSV文件列名

In [ ]:
#看df文件的列名
print(NewE_C_10_10 .columns)

# 模块一：得到目标论文序列

## STEP1——匹配所有引用了撤稿论文的文献序列df_containretra

In [ ]:
#这个部分需要两个原始数据
#（1）原始数据1：论文与其参考文献的对应关系（/Data1/DATA/Dimensions2024/20240101/PaperReferences.csv）
'''
df数据结构：
    id	            reference_ids
0	pub.1038660625	pub.1043519304
1	pub.1038660625	pub.1012015875
2	pub.1038660625	pub.1031233780
3	pub.1038660625	pub.1027495706
'''
#（2）原始数据2：4w篇撤稿论文的原始数据（/Data4/yutao_wen/matched_output2.csv）
'''
df数据结构（列名与数据类型）
Index(['UT', 'Paper_doi', 'Note_doi', 'Title', 'Journal', 'ISOAbbr',
       'Publisher', 'Article', 'Review', 'Institute', 'City', 'Country',
       'First', 'RP', 'Micro_topics', 'Meso_topics', 'Macro_topics', '3Years',
       '5Years', 'Total', 'Fund', 'Year', 'RYear', 'publish_date',
       'retraction_date', 'Open Access', 'Tags', 'Type', 'note_URL', 'Authors',
       'Addresses', 'Affiliations', 'PaperID', 'id', 'title', 'doi', 'date',
       'journal.id'],
      dtype='object')
'''


### 原始数据1与2的导入——30MIN

In [2]:
#先将两个原始文件都导入内核中
original_file1 = '/Data1/DATA/Dimensions2024/20240101/PaperReferences.csv'
df_references = pd.read_csv(original_file1)
original_file2 = '/Data4/yutao_wen/matched_output2.csv'
df_retrapapers = pd.read_csv(original_file2)
#time约30min

### 使用布尔索引匹配2个df——12MIN

In [3]:
# 保证reference_ids和撤稿论文id列的数据类型一致（通常为字符串）
df_references['reference_ids'] = df_references['reference_ids'].astype(str)
df_retrapapers['id'] = df_retrapapers['id'].astype(str)

# 使用布尔索引保留reference_ids出现在撤稿论文id中的行
df_containretra = df_references[df_references['reference_ids'].isin(df_retrapapers['id'])].copy()
#time约12min

### 匹配产出文件df_containretra预览

In [5]:
df_containretra

,id,reference_ids
0,pub.1148484199,pub.1013663776
1,pub.1128687208,pub.1003140261
2,pub.1148009542,pub.1146057724
3,pub.1114696738,pub.1026667060
4,pub.1114902564,pub.1028915017
...,...,...
765834,pub.1134201694,pub.1113603600
765835,pub.1016972058,pub.1008362454
765836,pub.1085464507,pub.1039593849
765837,pub.1124870274,pub.1124369222


### 匹配文件保存备份

In [6]:
output_path = '/Data4/yutao_wen/25.9.17/df_containretra.csv'
# 保存为 CSV 文件
df_containretra.to_csv(output_path, index=False)

### 导入df_containretra文件

In [29]:
df_containretra = pd.read_csv("/Data4/yutao_wen/25.9.17/df_containretra.csv")

## STEP2——匹配撤稿论文/目标论文原始信息

In [ ]:
#撤稿论文主要是两个关键年份（发表/撤稿）以及期刊id较为重要————来源于matched_output2
#（1）RPYear：被引用的撤稿论文的发表年份
#（2）RYear：被引用的撤稿论文的撤稿年份
#（3）RJournal_id：被引用的撤稿论文的期刊ID
#目标论文主要是发表年份与期刊id较为重要————来源于Papers.csv
#（1）Year：目标论文的发表年份
#（2）Journal_id：目标论文的期刊ID

### 原始数据2导入

In [30]:
original_file2 = '/Data4/yutao_wen/matched_output2.csv'
df_retrapapers = pd.read_csv(original_file2)

### 匹配撤稿论文RPYear、RYear、RJournal_id

In [31]:
#从 df_retrapapers 中选择需要匹配的列，并改列名
df_retra_subset = df_retrapapers[['id', 'Year', 'RYear', 'journal.id']].copy()
df_retra_subset = df_retra_subset.rename(columns={
    'id': 'reference_ids',
    'Year': 'RPYear',
    'RYear': 'RYear',
    'journal.id': 'RJournal_id'
})

# 第三步：将信息 merge 到 df_containretra 中
df_containretra = df_containretra.merge(df_retra_subset, on='reference_ids', how='left')

### 原始数据3导入——10min

In [11]:
# 此时需要第三个原始文件：Papers.csv文件
# 其蕴含了数据库所有论文的各种信息
original_file3 = '/Data1/DATA/Dimensions2024/20240101/Papers.csv'
'''
数据结构：
Index(['id', 'title', 'doi', 'date', 'date_inserted', 'date_normal',
       'date_online', 'date_print', 'type', 'journal.id', 'journal.title',
       'times_cited', 'num_funds', 'num_authors', 'num_references', 'field_l0',
       'field_l1', 'resulting_publication_doi'],
      dtype='object')

'''
#获取目标论文的Year与Journal.id
df_papers = pd.read_csv(original_file3,usecols=['id', 'date_normal','journal.id'])
#time约10min

### 详细年月日date提取年份Year

In [12]:
# 提取年份（ date 格式如 '2019-03-15'）
df_papers['Year'] = pd.to_datetime(df_papers['date_normal'], errors='coerce').dt.year

### 匹配目标论文Year、Journal.id——4MIN

In [32]:
# 只选择需要的列，重命名 'journal.id' 为 'Journal_id'
df_papers_subset = df_papers[['id', 'Year','journal.id']].rename(columns={'journal.id': 'Journal_id'})

# 合并，添加 Journal_id 列
df_containretra = df_containretra.merge(df_papers_subset, on='id', how='left')

In [36]:
# 调整列顺序
df_containretra = df_containretra[['id', 'reference_ids', 'Year', 'Journal_id', 'RPYear', 'RYear', 'RJournal_id']]


In [37]:
df_containretra

,id,reference_ids,Year,Journal_id,RPYear,RYear,RJournal_id
0,pub.1148484199,pub.1013663776,2019.0,NaN,2014.0,2022.0,jour.1044393
1,pub.1128687208,pub.1003140261,2012.0,NaN,2000.0,2012.0,jour.1412991
2,pub.1148009542,pub.1146057724,2022.0,jour.1115238,2022.0,2023.0,jour.1043255
3,pub.1114696738,pub.1026667060,2015.0,jour.1050605,2015.0,2016.0,jour.1084523
4,pub.1114902564,pub.1028915017,2012.0,jour.1038288,1992.0,2016.0,jour.1077219
...,...,...,...,...,...,...,...
766248,pub.1134201694,pub.1113603600,2020.0,jour.1041030,2019.0,2023.0,jour.1036281
766249,pub.1016972058,pub.1008362454,2016.0,jour.1046517,2013.0,2018.0,jour.1327387
766250,pub.1085464507,pub.1039593849,2017.0,jour.1454776,2011.0,2018.0,jour.1082971
766251,pub.1124870274,pub.1124369222,2020.0,jour.1319730,2020.0,2020.0,jour.1095007


### 检查每列NaN情况

In [38]:
# 计算每列的 NaN 数量
nan_counts = df_containretra.isna().sum()

# 输出每列 NaN 的数量
print("每列 NaN 的数量：")
print(nan_counts)


每列 NaN 的数量：
id                   0
reference_ids        0
Year                84
Journal_id       71649
RPYear            3985
RYear             4786
RJournal_id      25646
dtype: int64


### 保留不含NaN的行

In [39]:
# 计算删除前 DataFrame 的总行数
total_rows_before = len(df_containretra)

# 删除含有 NaN 的行
df_containretra_cleaned = df_containretra.dropna(axis=0, how='any')

# 计算删除后每列的 NaN 数量
nan_counts_after = df_containretra_cleaned.isna().sum()

# 计算删除后的 DataFrame 的总行数
total_rows_after = len(df_containretra_cleaned)

# 计算删除的行数
rows_deleted = total_rows_before - total_rows_after

# 输出结果
print("\n删除了多少行（含有 NaN 的行）：", rows_deleted)
print("\n删除后每列 NaN 的数量：")
print(nan_counts_after)
print("\n删除前的行数：", total_rows_before)
print("删除后的行数：", total_rows_after)


删除了多少行（含有 NaN 的行）： 96608

删除后每列 NaN 的数量：
id               0
reference_ids    0
Year             0
Journal_id       0
RPYear           0
RYear            0
RJournal_id      0
dtype: int64

删除前的行数： 766253
删除后的行数： 669645


In [40]:
df_containretra_cleaned

,id,reference_ids,Year,Journal_id,RPYear,RYear,RJournal_id
2,pub.1148009542,pub.1146057724,2022.0,jour.1115238,2022.0,2023.0,jour.1043255
3,pub.1114696738,pub.1026667060,2015.0,jour.1050605,2015.0,2016.0,jour.1084523
4,pub.1114902564,pub.1028915017,2012.0,jour.1038288,1992.0,2016.0,jour.1077219
7,pub.1107325835,pub.1003143127,2018.0,jour.1030157,2006.0,2018.0,jour.1030157
8,pub.1011260253,pub.1032808220,2008.0,jour.1013016,2005.0,2007.0,jour.1013016
...,...,...,...,...,...,...,...
766247,pub.1130857724,pub.1117049318,2020.0,jour.1136301,2019.0,2023.0,jour.1138382
766248,pub.1134201694,pub.1113603600,2020.0,jour.1041030,2019.0,2023.0,jour.1036281
766249,pub.1016972058,pub.1008362454,2016.0,jour.1046517,2013.0,2018.0,jour.1327387
766250,pub.1085464507,pub.1039593849,2017.0,jour.1454776,2011.0,2018.0,jour.1082971


### 匹配文件保存与备份

In [41]:
output_path = '/Data4/yutao_wen/25.9.17/df_containretra2.csv'
# 保存为 CSV 文件
df_containretra_cleaned.to_csv(output_path, index=False)
'''
数据结构为：
	            id	reference_ids	Year	 Journal_id	    RPYear	RYear	RJournal_id
2	pub.1148009542	pub.1146057724	2022.0	jour.1115238	2022.0	2023.0	jour.1043255
3	pub.1114696738	pub.1026667060	2015.0	jour.1050605	2015.0	2016.0	jour.1084523
4	pub.1114902564	pub.1028915017	2012.0	jour.1038288	1992.0	2016.0	jour.1077219
7	pub.1107325835	pub.1003143127	2018.0	jour.1030157	2006.0	2018.0	jour.1030157
8	pub.1011260253	pub.1032808220	2008.0	jour.1013016	2005.0	2007.0	jour.1013016
...	...	...	...	...	...	...	...
'''

'\n数据结构为：\n\t            id\treference_ids\tYear\t Journal_id\t    RPYear\tRYear\tRJournal_id\n2\tpub.1148009542\tpub.1146057724\t2022.0\tjour.1115238\t2022.0\t2023.0\tjour.1043255\n3\tpub.1114696738\tpub.1026667060\t2015.0\tjour.1050605\t2015.0\t2016.0\tjour.1084523\n4\tpub.1114902564\tpub.1028915017\t2012.0\tjour.1038288\t1992.0\t2016.0\tjour.1077219\n7\tpub.1107325835\tpub.1003143127\t2018.0\tjour.1030157\t2006.0\t2018.0\tjour.1030157\n8\tpub.1011260253\tpub.1032808220\t2008.0\tjour.1013016\t2005.0\t2007.0\tjour.1013016\n...\t...\t...\t...\t...\t...\t...\t...\n'

## STEP3——筛选出合理目标论文

In [42]:
#主要是对STEP2的数据进行无关对象清除
#因为我们关注的目标论文是：
# （1）参考文献仅仅有一篇被撤稿
# （2）而且撤稿时间在2023年之前
# （3）引用论文的时候该被引论文尚未被撤稿
# （4）自己本身没被撤稿
#Exclude data ————— 主要有四部分被去掉
#(1) cited more than one retracted paper
#(2) retracted in 2023 or 2024
#(3) citing papers that were themselves retracted
#(4) The paper itself was retracted

### 导入STEP2终文件df_containretra2

In [43]:
df_containretra2 = '/Data4/yutao_wen/25.9.17/df_containretra2.csv'
df_containretra2 = pd.read_csv(df_containretra2)

### （1）删除参考文献有一篇以上被撤稿的

In [44]:
#(1)cited more than one retracted paper
#需要先去掉参考文献被撤稿了2篇及以上的目标论文，因为这部分的撤稿冲击存在多个时间点，分析复杂，暂不考虑
df_containretra_copy = df_containretra2.copy()

# 统计原始行数
original_count = len(df_containretra_copy)

# 删除所有出现两次及以上的 id 的行
df_containretra_copy = df_containretra_copy[~df_containretra_copy['id'].duplicated(keep=False)]

# 统计删减后行数
filtered_count = len(df_containretra_copy)

# 输出对比信息
print(f"原来有 {original_count} 行")
print(f"去除重复 id 后有 {filtered_count} 行")
print(f"共删除了 {original_count - filtered_count} 行")

原来有 669645 行
去除重复 id 后有 499051 行
共删除了 170594 行


### （2）删除参考文献在2023、2024被撤稿的

In [45]:
#(2)retracted in 2023 or 2024
#去掉2023年、2024年被撤稿的参考文献部分
#因为需要研究撤稿后的趋势，这两年撤稿的文章后续时期的数据太少了，不便于分析
# 创建副本，避免修改原始数据

# 记录原始行数
original_count = len(df_containretra_copy)

# 去掉 RPYear 为 2023 或 2024 的行
df_containretra_copy = df_containretra_copy[~df_containretra_copy['RYear'].isin([2023.0, 2024.0])]

# 记录删减后行数
filtered_count = len(df_containretra_copy)

# 输出行数信息
print(f"原来有 {original_count} 行")
print(f"删减后有 {filtered_count} 行")
print(f"共删除了 {original_count - filtered_count} 行")

原来有 499051 行
删减后有 363465 行
共删除了 135586 行


### （3）删除引用参考文献时就已被撤稿的

In [46]:
#(3)citing papers that were themselves retracted
#发现存在有的论文引用了已经被撤稿的论文，数学表现形式即———— RYear-Year < 0，需要删除
# 创建新列 RYear-Year
df_containretra_copy['RYear-Year'] = df_containretra_copy['RYear'] - df_containretra_copy['Year']

# 记录原始行数
original_count = len(df_containretra_copy)

# 删除 RYear-Year 小于 0 的行
df_containretra_copy = df_containretra_copy[df_containretra_copy['RYear-Year'] >= 0]

# 删除后行数
remaining_count = len(df_containretra_copy)

# 打印统计信息
print(f"原来有 {original_count} 行")
print(f"删除 RYear-Year < 0 的行数：{original_count - remaining_count}")
print(f"剩余 {remaining_count} 行")

原来有 363465 行
删除 RYear-Year < 0 的行数：143452
剩余 220013 行


### （4）删除自己本身也被撤稿的论文

In [47]:
#(4)The paper itself was retracted
#还有本身自己就被撤稿的论文
#检查 df_containretra_copy的id列中的id是否有出现在那4w篇撤稿文章id（df_retrapapers）中
#如果出现了就需要删除这些行
# 统计原始行数
original_count = len(df_containretra_copy)

# 删除df_containretra_copy中id在撤稿论文id列表中的行
df_containretra_copy = df_containretra_copy[~df_containretra_copy['id'].isin(df_retrapapers['id'])]

# 统计删减后行数
filtered_count = len(df_containretra_copy)

# 输出统计信息
print(f"原来有 {original_count} 行")
print(f"删除本身被撤稿的论文后，剩余 {filtered_count} 行")
print(f"共删除了 {original_count - filtered_count} 行")

原来有 220013 行
删除本身被撤稿的论文后，剩余 217296 行
共删除了 2717 行
